In [1]:
# 把函数视为对象
def factorial(n):
    """return n!"""
    return 1 if n < 2 else n * factorial(n - 1)

In [2]:
# __doc__是函数对象众多属性的一个
factorial.__doc__

'return n!'

In [3]:
type(factorial)

function

In [4]:
# 通过别的名称使用函数
fact = factorial
fact

<function __main__.factorial(n)>

In [5]:
fact(5)

120

In [6]:
# 函数可以作为参数传入另一个函数
map(factorial, range(11))

In [7]:
list(map(factorial, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [8]:
# 高阶函数
# 接受函数为参数，或者返回值为函数的函数就是高阶函数
# 内置函数sorted就是一个高阶函数，可选参数key可以接受一个函数对象
# 它会应用到各个元素上进行排序
# 根据单词的长度给列表排序
fruits = ['fig', 'apple', 'cherry', 'raspberry', 'banana', 'strawberry']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [9]:
# lambda - 匿名函数
# 在参数列表中最适合使用匿名函数
sorted(fruits, key=lambda x: x[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

In [10]:
# callable - 可调用对象
# 除用户定义的函数外，调用运算符（即()）还可以应用到其他对象上
# 可以使用callable()函数判断一个对象是否可以调用
# 以下7种对象是可调用对象：
# 1、用户定义的函数 - 使用def语句或lambda表达式创建
# 2、内置函数 - 使用CPyhton实现的函数
# 3、方法 - 在类定义体中定义的函数
# 4、类
# 5、类的实例
# 6、生成器函数
[callable(obj) for obj in [str, abs, 13]]

[True, True, False]

In [11]:
# 用户定义的可调用类型
# 只要实现实例方法__call__，任何对象都可以表现得像函数
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    # bingo.pick()的快捷方式就是bingo()
    def __call__(self):
        return self.pick()

In [12]:
bingo = BingoCage(range(3))
bingo.pick()

0

In [13]:
bingo()

2

In [14]:
callable(bingo)

True

In [4]:
# keyword-only argument - 仅限关键字参数
def tag(name, *content, cls=None, **attrs):
    """生成一个或多个HTML标签"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                            for attr, value 
                            in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % 
                        (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [5]:
# tag函数的众多调用方式
# 传入单个定位参数 - name = 'bar'
tag('bar')

'<bar />'

In [7]:
# 第一个参数后面的任意个参数会被*content捕获，存入一个元组
tag('p', 'hello')

'<p>hello</p>'

In [9]:
print(tag('p', 'hello', 'world'))

<p>hello</p>
<p>world</p>


In [10]:
# 函数中没有明确指定名称的关键字参数会被**attrs捕获，存入一个字典
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [11]:
# cls参数只能作为关键字参数传入
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [12]:
# 调用tag函数时，即便第一个定位参数也能作为关键字参数传入
tag(content='testing', name='img')

'<img content="testing" />'

In [13]:
my_tags = {
    'name': 'img',
    'title': 'Sunshine',
    'src': 'sunset.jpg',
    'cls': 'framed'
}

# 在my_tags前加**，字典中的所有元素作为单个参数传入
# 同名键会绑定到对应的具名参数上
# 余下的元素会被**attrs捕获
tag(**my_tags)

'<img class="framed" src="sunset.jpg" title="Sunshine" />'

In [14]:
# 函数注解 - typehint
def clip(text: str, max_len: 'int > 0' =80) -> str:
    """在max_len前面或后面的第一个空格处截断文本"""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
        if end is None:
            end = len(text)
        return text[:end].rstrip()

In [16]:
# 设计模式 - 策略
# 实现order类，支持插入式折扣策略
from abc import ABC, abstractclassmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order: # 上下文
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

class Promotion(ABC): # 策略：抽象基类

    @abstractclassmethod
    def discount(self, order):
        """返回折扣金额（正值）"""

class FidelityPromo(Promotion): # 第一个具体策略
    """为积分1000或以上的顾客提供5%的折扣"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion): # 第二个具体策略
    """单个商品为20个以上的提供10%折扣"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion): # 第三个具体策略
    """订单中不同商品数到10或以上时提供7%折扣"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [17]:
# 测试数据
joe = Customer('joe', 0)
ann = Customer('ann', 1100)

cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)
]

Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [18]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [20]:
banana_cart = [
    LineItem('banana', 30, .5),
    LineItem('apple', 10, 1.5)
    ]

Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [22]:
long_order = [LineItem(str(item), 1, 1.0)
                for item in range(10)]

Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [25]:
# Order类和使用函数实现的折扣策略
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.quantity * self.price


class Order: # 上下文
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):
    """为积分为1000或以上的顾客提供5%折扣"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """单个商品为20个或以上时提供10%折扣"""
    discount = 0
    for itme in order.cart:
        if itme.quantity >= 20:
            discount += itme.total() * .1
    return discount

def large_order_promo(order):
    """订单中的不同商品达到10个或以上时提供7%折扣"""
    distinct_items = {item for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0 


In [28]:
joe = Customer('joe', 0)
ann = Customer('ann', 1100)

cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)
]

Order(joe, cart, fidelity_promo)

<Order: 42.00 due: 42.00>

In [27]:
Order(ann, cart, fidelity_promo)

<Order: 42.00 due: 39.90>

In [29]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

Order(joe, banana_cart, bulk_item_promo)

<Order: 30.00 due: 28.50>

In [30]:
long_order = [LineItem(str(item_code), 1, 1.0)
                for item_code in range(10)]

Order(joe, long_order, large_order_promo)

<Order: 10.00 due: 9.30>

In [31]:
# 装饰器
# 装饰器是可调用的对象，其参数时另一个函数
# 装饰器可能会处理被装饰的函数，然后将它返回
# 或者将其替换为另一函数或可调用对象
def deco(func):
    def inner():
        print('run inner()')
    return inner

@deco
def target():
    print('run target()') # 等价于target = deco(target)

target()

run inner()


In [32]:
target

<function __main__.deco.<locals>.inner()>

In [34]:
# 装饰器的一个关键特性是，它们在被装饰的函数定义之后立即运行。这 
# 通常是在导入时 （即 Python 加载模块时）
# 保存被@register装饰的函数引用
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func


@register
def f1():
    print('running f1()')


@register
def f2():
    print('running f2()')


@register
def f3():
    print('running f3()')


def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()


if __name__ == '__main__':
    main()

running register(<function f1 at 0x000001EF79C1A940>)
running register(<function f2 at 0x000001EF79C1A790>)
running register(<function f3 at 0x000001EF79C1AA60>)
running main()
registry -> [<function f1 at 0x000001EF79C1A940>, <function f2 at 0x000001EF79C1A790>, <function f3 at 0x000001EF79C1AA60>]
running f1()
running f2()
running f3()


In [35]:
# 使用装饰器改进策略模式
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func


@promotion
def fidelity_promo(order):
    """为积分为1000或以上的顾客提供5%折扣"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


@promotion
def bulk_item_promo(order):
    """单个商品为20个或以上时提供10%折扣"""
    discount = 0
    for itme in order.cart:
        if itme.quantity >= 20:
            discount += itme.total() * .1
    return discount


@promotion
def large_order_promo(order):
    """订单中的不同商品达到10个或以上时提供7%折扣"""
    distinct_items = {item for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0 


def best_promo(order):
    """选择可用的最佳折扣"""
    return max(promo(order) for promo in promos)

In [36]:
# 变量作用域规则
# 一个函数，读取一个局部变量和全局变量
def f1(a):
    print(a)
    print(b)

# 出现错误，因为变量b没有定义
f1(3)

3


NameError: name 'b' is not defined

In [37]:
b = 6 
f1(3)

3
6


In [38]:
b = 3
def f2(a):
    print(a)
    print(b)
    b = 9

# Python 编译函数的定义体时，它判断 b 是局部变量，因为在函数中给它赋值了
# 后面调用 f2(3) 时， f2 的定义体会获取并打印局部变量 a 的值，但是尝试获取局部变量 b 的值时，发现 b 没有绑定值。
f2(6)

6


UnboundLocalError: local variable 'b' referenced before assignment

In [40]:
# 如果在函数中赋值时想让解释器把 b 当成全局变量，要使用 global 声明
b = 3
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(6)

6
3


In [41]:
b

9

In [1]:
# 闭包 - 延伸了作用域的函数
# 其中包含函数定义体中引用、但不在定义体中定义的非全局变量
# 普通的计算移动平均的类
class  Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [3]:
# Averager类的实例是可调用的（callable）
avg = Averager()

In [4]:
avg(10)

10.0

In [5]:
avg(11)

10.5

In [6]:
avg(12)

11.0

In [7]:
avg.series

[10, 11, 12]

In [8]:
# 使用闭包函数实现一个移动平均计算函数
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

In [9]:
# 调用make_averager时，返回一个averager函数对象
avg = make_averager()
avg

<function __main__.make_averager.<locals>.averager(new_value)>

In [10]:
type(avg)

function

In [11]:
avg(10)

10.0

In [12]:
avg(11)

10.5

In [13]:
avg(12)

11.0

In [14]:
# 不保存所有历史值，来计算移动平均值
def make_averager():

    count = 0
    total = 0

    def averager(new_value):
        # 对于不可变类型，闭包只能读取，不能更新
        # 下面做法会隐式的将count、total转化为局部变量
        count += 1
        total += new_value
        return total / count

    return averager

In [15]:
avg = make_averager()

In [16]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [17]:
# 解决方法 - nolocal
def make_averager():

    count = 0
    total = 0

    def averager(new_value):
        # 使用nolocal关键声明count, total并非局部变量
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [18]:
avg = make_averager()
avg(10)

10.0

In [22]:
# 一个简单的装饰器 - 输出函数的运行时间
import time

def clock(func):
    # 定义内部函数，它接受任意个定位参数
    def clocked(*args):
        t0 = time.perf_counter()
        # clocked闭包包含自由变量func
        res = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, res))
        return res
    # 返回内部函数，取代被装饰的函数
    return clocked

In [29]:
@clock
def snooze(second):
    time.sleep(second)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [25]:
snooze(.123)

[0.12245750s] snooze(0.123) -> None


In [30]:
factorial(6)

[0.00000040s] factorial(1) -> 1
[0.00003040s] factorial(2) -> 2
[0.00004090s] factorial(3) -> 6
[0.00005090s] factorial(4) -> 24
[0.00006140s] factorial(5) -> 120
[0.00007080s] factorial(6) -> 720


720

In [31]:
# 改进
# 一个简单的装饰器 - 输出函数的运行时间
import time
import functools

def clock(func):
    # 定义内部函数，它接受任意个定位参数
    # wraps装饰器将有关属性从func复制给clocked
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        # clocked闭包包含自由变量func
        res = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, res))
        return res
    # 返回内部函数，取代被装饰的函数
    return clocked

In [32]:
# 使用functools.lru_cache做备忘
# 生成第n个斐波那契数，递归方式非常耗时
# fibonacci(1)调用了8次，fibonacci(2)调用了5次...
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(6)

[0.00000050s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00013290s] fibonacci(2) -> 1
[0.00000020s] fibonacci(1) -> 1
[0.00000020s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00001740s] fibonacci(2) -> 1
[0.00003510s] fibonacci(3) -> 2
[0.00018610s] fibonacci(4) -> 3
[0.00000020s] fibonacci(1) -> 1
[0.00000020s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00001760s] fibonacci(2) -> 1
[0.00003440s] fibonacci(3) -> 2
[0.00000010s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00001730s] fibonacci(2) -> 1
[0.00000020s] fibonacci(1) -> 1
[0.00000030s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00001790s] fibonacci(2) -> 1
[0.00003530s] fibonacci(3) -> 2
[0.00006920s] fibonacci(4) -> 3
[0.00012060s] fibonacci(5) -> 5
[0.00032390s] fibonacci(6) -> 8


8

In [33]:
# 使用缓存实现
import functools

# 叠放多个装饰器
# 等价于 finonacci=functools.lru_cache(clock(fibonacci))
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

# 时间减少一半，且n的每个值只调用一次函数
fibonacci(6)

[0.00000040s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00006520s] fibonacci(2) -> 1
[0.00000060s] fibonacci(3) -> 2
[0.00009600s] fibonacci(4) -> 3
[0.00000070s] fibonacci(5) -> 5
[0.00012560s] fibonacci(6) -> 8


8

In [34]:
# 参数化装饰器
registry = set()

def registor(active=True):
    def decorate(func):
        print('running register(active=%s) -> decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@registor(active=False)
def f1():
    print('runing f1()')

@registor()
def f2():
    print('running f2()')

def f3():
    print('runing f3()')

running register(active=False) -> decorate(<function f1 at 0x000001E2B83E69D0>)
running register(active=True) -> decorate(<function f2 at 0x000001E2B83E6040>)


In [35]:
registry

{<function __main__.f2()>}

In [36]:
registor()(f3)

running register(active=True) -> decorate(<function f3 at 0x000001E2B83E6C10>)


<function __main__.f3()>

In [37]:
registor(active=False)(f2)

running register(active=False) -> decorate(<function f2 at 0x000001E2B83E6040>)


<function __main__.f2()>

In [38]:
registry

{<function __main__.f3()>}